In [8]:
import pandas as pd
import tensorflow as tf
file_path = r'C:\Users\Public\5th\NeuralNetworks\ASSGINMENT3\Ex01\default of credit card clients.xls'
df = pd.read_excel(file_path)

print(df.isnull().sum())
print(df.dtypes)
df = df.drop(columns=['ID'])

export_file_path = r'C:\Users\Public\5th\NeuralNetworks\ASSGINMENT3\Ex01\default_of_credit_card_clients.csv'

df.to_csv(export_file_path, index=False)

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAYDELAY_MONTH1               0
PAYDELAY_MONTH2               0
PAYDELAY_MONTH3               0
PAYDELAY_MONTH4               0
PAYDELAY_MONTH5               0
PAYDELAY_MONTH6               0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64
ID                             int64
LIMIT_BAL                      int64
SEX                           object
EDUCATION                     object
MARRIAGE                      object
AG

In [9]:
dataset = tf.data.experimental.make_csv_dataset(
    export_file_path,
    batch_size=32,  
    label_name='default payment next month'
)

for features, labels in dataset.take(1):
    print("Features:", features)
    print("Labels:", labels)

Features: OrderedDict([('LIMIT_BAL', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([130000, 300000,  30000, 100000, 500000,  10000, 200000, 670000,
        80000, 250000, 230000,  30000,  50000,  60000, 200000, 100000,
        80000,  80000,  50000,  30000, 140000, 120000,  20000,  50000,
       110000,  50000,  30000, 240000,  20000,  90000, 260000,  80000])>), ('SEX', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'M', b'F', b'F', b'M', b'F', b'M', b'M', b'M', b'F', b'F', b'F',
       b'F', b'F', b'F', b'M', b'M', b'F', b'F', b'M', b'F', b'F', b'F',
       b'M', b'F', b'F', b'M', b'M', b'F', b'M', b'M', b'F', b'M'],
      dtype=object)>), ('EDUCATION', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'graduate school', b'graduate school', b'university',
       b'university', b'high school', b'university', b'graduate school',
       b'graduate school', b'university', b'graduate school',
       b'high school', b'high school', b'graduate school', b'university',
       b

In [10]:
# converting categorical to numerical
def encode_categorical(features, labels):
    sex_mapping = {'M': 0, 'F': 1}
    education_mapping = {'university': 0, 'graduate school': 1, 'high school': 2, 'others': 3}
    marriage_mapping = {'single': 0, 'married': 1, 'others': 2}
    
    # key-value mappings for each categorical feature.
    sex_initializer = tf.lookup.KeyValueTensorInitializer(keys=list(sex_mapping.keys()), values=list(sex_mapping.values()), key_dtype=tf.string, value_dtype=tf.int64)
    education_initializer = tf.lookup.KeyValueTensorInitializer(keys=list(education_mapping.keys()), values=list(education_mapping.values()), key_dtype=tf.string, value_dtype=tf.int64)
    marriage_initializer = tf.lookup.KeyValueTensorInitializer(keys=list(marriage_mapping.keys()), values=list(marriage_mapping.values()), key_dtype=tf.string, value_dtype=tf.int64)
    
    # use the hash tables to convert the categorical string values in the features
    sex_table = tf.lookup.StaticHashTable(sex_initializer, default_value=-1)
    education_table = tf.lookup.StaticHashTable(education_initializer, default_value=-1)
    marriage_table = tf.lookup.StaticHashTable(marriage_initializer, default_value=-1)

    features['SEX'] = sex_table.lookup(features['SEX'])
    features['EDUCATION'] = education_table.lookup(features['EDUCATION'])
    features['MARRIAGE'] = marriage_table.lookup(features['MARRIAGE'])

    return features, labels

dataset = dataset.map(encode_categorical)

for features, labels in dataset.take(1):  
    print("Features:", features)
    print("Labels:", labels)

Features: OrderedDict([('LIMIT_BAL', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([280000, 120000, 140000, 210000, 100000, 150000, 350000,  30000,
       200000, 320000, 280000, 300000, 360000,  30000, 360000, 320000,
       300000, 310000,  80000, 280000, 220000, 230000, 370000, 500000,
       170000,  70000, 200000, 400000, 250000, 200000, 240000, 160000])>), ('SEX', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1], dtype=int64)>), ('EDUCATION', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0], dtype=int64)>), ('MARRIAGE', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1], dtype=int64)>), ('AGE', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([28, 26, 24, 31, 28,

In [11]:
# transformation
def square_features(features, labels):
    for feature, value in features.items():
        features[feature] = tf.square(value)
    return features, labels

dataset = dataset.map(square_features)

for features, labels in dataset.take(1):  
    print("Features:", features)
    print("Labels:", labels)

Features: OrderedDict([('LIMIT_BAL', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([ 2105032704,  -169803776, -1628888064,   900000000, -1794967296,
       -1164771072, -1924509440,  -679215104,   400000000, -1959738368,
        1610719488, -1794967296,  -279869184,  2105032704, -1794967296,
        1610719488,  -489934592,   900000000,  -489934592,  1150327040,
        1765425152, -1794967296, -1959738368,  1515098112,  2105032704,
        -538953472,   900000000,   605032704,  2105032704, -1799345920,
       -1794967296,  2105032704])>), ('SEX', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)>), ('EDUCATION', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([4, 1, 1, 1, 4, 1, 4, 0, 0, 1, 4, 0, 4, 0, 4, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 4, 4, 4], dtype=int64)>), ('MARRIAGE', <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 0, 0, 0, 0, 0, 

In [12]:
# standardization
def standardize_features(features, labels):
    for feature_name in features.keys():
        feature = tf.cast(features[feature_name], tf.float32)
        
        mean, variance = tf.nn.moments(feature, axes=0)
        std_dev = tf.sqrt(variance)
        features[feature_name] = (feature - mean) / std_dev
    
    return features, labels

dataset = dataset.map(standardize_features)

for features, labels in dataset.take(1):  
    print("Features:", features)
    print("Labels:", labels)

Features: OrderedDict([('LIMIT_BAL', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 0.03803366,  0.43912873,  0.8088313 , -1.7517533 ,  0.8616459 ,
        0.601143  , -0.58503103,  1.4283257 , -0.02192168, -1.7517533 ,
       -1.7517533 ,  1.1308895 , -1.8861084 , -0.4265871 ,  0.65395766,
        0.44573608, -0.38144648,  0.03803366,  0.20521833,  0.8088313 ,
        1.4283257 , -1.7517533 ,  0.03803366, -1.7517533 ,  0.43912873,
        1.4283257 ,  0.54779506,  0.8088313 ,  0.601143  , -0.4465722 ,
       -0.20658779, -0.03833645], dtype=float32)>), ('SEX', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 0.82717025, -1.2089411 ,  0.82717025, -1.2089411 , -1.2089411 ,
       -1.2089411 ,  0.82717025, -1.2089411 ,  0.82717025,  0.82717025,
       -1.2089411 , -1.2089411 ,  0.82717025, -1.2089411 ,  0.82717025,
       -1.2089411 ,  0.82717025,  0.82717025,  0.82717025,  0.82717025,
        0.82717025, -1.2089411 ,  0.82717025, -1.2089411 , -1.2089411 ,
        0.8271702